In [139]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)




In [140]:
# conda activate aso_design
import pandas as pd
from scripts.data_genertion.consts import *
from asodesigner.file_utils import read_human_genome_fasta_dict
from asodesigner.consts import *
all_data = pd.read_csv(DATA_PATH / 'data_asoptimizer_updated.csv')
all_data.head()

/tmp/ipykernel_235140/1107691422.py:6: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv(DATA_PATH / 'data_asoptimizer_updated.csv')


,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Inhibition(%),seq_length,Canonical Gene Name,Cell line organism,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform
0,0,540733,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCTAAAACAAATGCTA,...,33.0,16,KRAS,human,NaN,41212.0,0.901972,16,0,A431
1,1,540747,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,TATAATGGTGAATATC,...,7.0,16,KRAS,human,NaN,23686.0,0.518395,16,0,A431
2,2,540806,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCATGAAGATTTCTGG,...,62.0,16,KRAS,human,NaN,43363.0,0.949049,16,1,A431
3,3,651479,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GGTGAATATCTTCAAA,...,28.0,16,KRAS,human,NaN,23680.0,0.518264,16,0,A431
4,4,651490,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,CACTTGTACTAGTATG,...,36.0,16,KRAS,human,NaN,41168.0,0.901009,16,0,A431


In [141]:
from scipy.stats import pearsonr, spearmanr
from sklearn.feature_selection import mutual_info_regression
def print_correlations(df, name1, name2, p_value_threshold=None):
    if p_value_threshold is None:
        p_value_threshold = 1
    corr, p_value = pearsonr(df[name1], df[name2])
    if p_value < p_value_threshold:
        print(f"Feature: {name1:<35}, Pearson: {corr:<5.2f}, p-value: {p_value:<10.2} Target: {name2:<35}")
    corr, p_value = spearmanr(df[name1], df[name2])
    if p_value < p_value_threshold:
        print(f"Feature: {name1:<35}, Spearman: {corr:<5.2f}, p-value: {p_value:<10.2} Target: {name2:<35}")

        # MIC (via Mutual Information)
    try:
        mic = mutual_info_regression(df[[name1]], df[name2], discrete_features='auto')[0]
        print(f"Feature: {name1:<35}, MIC:      {mic:<5.2f}                             Target: {name2:<35}")
    except Exception as e:
        print(f"Could not compute MIC for {name1} vs {name2}: {e}")    

Feture generation

In [142]:
# purine content
from asodesigner.features.seq_features import purine_content
from Bio.Seq import Seq

all_data['ASO_purine_content'] = all_data["Sequence"].apply(lambda x: purine_content(x))
sequence_features = ['ASO_purine_content']

In [143]:
from Bio.SeqUtils import gc_fraction

all_data['ASO_gc_content'] = all_data["Sequence"].apply(gc_fraction)
sequence_features.append('ASO_gc_content')


In [144]:
# count_g_runs
from asodesigner.features.seq_features import count_g_runs
all_data['ASO_ggg_counts'] = all_data["Sequence"].apply(lambda x: count_g_runs(x))
sequence_features.append("ASO_ggg_counts")

In [145]:
#ENC
from asodesigner.features.seq_features import compute_ENC
all_data["ASO_ENC"] = all_data["Sequence"].apply(lambda x: compute_ENC(x))
sequence_features.append("ASO_ENC")

In [146]:
# palindromic count of 4 and 6 palindroms
from asodesigner.features.seq_features import palindromic_fraction
all_data['ASO_4 palindromic'] = all_data[SEQUENCE].apply(lambda x: palindromic_fraction(x,4))
sequence_features.append("ASO_4 palindromic")

all_data['ASO_6 palindromic'] = all_data[SEQUENCE].apply(lambda x: palindromic_fraction(x,6))
sequence_features.append("ASO_6 palindromic")


In [147]:
# entropy
from asodesigner.features.seq_features import seq_entropy
all_data["ASO_entropy"] = all_data["Sequence"].apply(lambda x: seq_entropy(x))
sequence_features.append("ASO_entropy")

In [148]:
#hairpin_score
from asodesigner.features.seq_features import hairpin_score
all_data["ASO_hairpin_score"] = all_data["Sequence"].apply(lambda x : hairpin_score(x))
sequence_features.append("ASO_hairpin_score")

In [149]:
#toxic_motif_count
from asodesigner.features.seq_features import toxic_motif_count
all_data["ASO_toxic_motif_count"] = all_data["Sequence"].apply(lambda x : toxic_motif_count(x))
sequence_features.append("ASO_toxic_motif_count")

In [150]:
# hairpin_dG_energy
from asodesigner.features.seq_features import hairpin_dG_energy
all_data["ASO_hairpin_dG_energy"] = all_data["Sequence"].apply(lambda x : hairpin_dG_energy(x))
sequence_features.append("ASO_hairpin_dG_energy")

structure_found: False
structure_found: False
structure_found: False
structure_found: True
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: True
structure_found: True
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: True
structure_found: True
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found: False
structure_found:

In [151]:
#hairpin_tm
from asodesigner.features.seq_features import hairpin_tm
all_data["ASO_hairpin_tm"] = all_data["Sequence"].apply(lambda x : hairpin_tm(x))
sequence_features.append("ASO_hairpin_tm")


In [152]:
#tandem_repeats_score
from asodesigner.features.seq_features import tandem_repeats_score
all_data["ASO_tandem_repeats_score"] = all_data["Sequence"].apply(lambda x : tandem_repeats_score(x))
sequence_features.append("ASO_tandem_repeats_score")

In [153]:
# dispersed_repeats_score
from asodesigner.features.seq_features import dispersed_repeats_score
all_data["ASO_dispersed_repeats_score"] = all_data["Sequence"].apply(lambda x : dispersed_repeats_score(x))
sequence_features.append("ASO_dispersed_repeats_score")


In [154]:
#flexible_dinucleotide_fraction
from asodesigner.features.seq_features import flexible_dinucleotide_fraction
all_data["ASO_flexible_dinucleotide_fraction"] = all_data["Sequence"].apply(lambda x : flexible_dinucleotide_fraction(x))
sequence_features.append("ASO_flexible_dinucleotide_fraction")

In [155]:
#gc_skew
from asodesigner.features.seq_features import gc_skew
all_data["ASO_gc_skew"] = all_data["Sequence"].apply(lambda x : gc_skew(x))
sequence_features.append("ASO_gc_skew")

In [156]:
#gc_skew_ends
from asodesigner.features.seq_features import gc_skew_ends
all_data["ASO_gc_skew_ends"] = all_data["Sequence"].apply(lambda x : gc_skew_ends(x))
sequence_features.append("ASO_gc_skew_ends")

In [157]:
#at_skew
from asodesigner.features.seq_features import at_skew
all_data["ASO_at_skew"] = all_data["Sequence"].apply(lambda x : at_skew(x))
sequence_features.append("ASO_at_skew")

In [158]:
#stop_codon_count
from asodesigner.features.seq_features import stop_codon_count
all_data["ASO_stop_codon_count"] = all_data["Sequence"].apply(lambda x : stop_codon_count(x))
sequence_features.append("ASO_stop_codon_count")

In [159]:
#nucleotide_diversity
from asodesigner.features.seq_features import nucleotide_diversity
all_data["ASO_nucleotide_diversity"] = all_data["Sequence"].apply(lambda x : nucleotide_diversity(x))
sequence_features.append("ASO_nucleotide_diversity")

In [160]:
#cg_dinucleotide_fraction
import importlib
#import niv_features
#importlib.reload(niv_features)
from asodesigner.features.seq_features import cg_dinucleotide_fraction
all_data["ASO_cg_dinucleotide_fraction"] = all_data["Sequence"].apply(lambda x : cg_dinucleotide_fraction(x))
sequence_features.append("ASO_cg_dinucleotide_fraction")

In [161]:
#poly_pyrimidine_stretch
from asodesigner.features.seq_features import poly_pyrimidine_stretch
all_data["ASO_poly_pyrimidine_stretch"] = all_data["Sequence"].apply(lambda x : poly_pyrimidine_stretch(x))
sequence_features.append("ASO_poly_pyrimidine_stretch")

In [162]:
#dinucleotide_entropy
from asodesigner.features.seq_features import dinucleotide_entropy
all_data["ASO_dinucleotide_entropy"] = all_data["Sequence"].apply(lambda x : dinucleotide_entropy(x))
sequence_features.append("ASO_dinucleotide_entropy")

In [163]:
#gc_block_length
from asodesigner.features.seq_features import gc_block_length
all_data["ASO_gc_block_length"] = all_data["Sequence"].apply(lambda x : gc_block_length(x))
sequence_features.append("ASO_gc_block_length")

In [164]:
#at_rich_region_score
from asodesigner.features.seq_features import at_rich_region_score
all_data["ASO_at_rich_region_score"] = all_data["Sequence"].apply(lambda x : at_rich_region_score(x))
sequence_features.append("ASO_at_rich_region_score")

In [165]:
from asodesigner.features.tai import calc_tAI, tai_weights
weights_tai = tai_weights("hm")
all_data["ASO_tai"] = all_data["Sequence"].apply(lambda x : calc_tAI(x, weights_tai))
sequence_features.append("ASO_tai")

In [166]:
from asodesigner.features.seq_features import gc_content_3prime_end
all_data["ASO_gc_content_3prime_end"] = all_data["Sequence"].apply(lambda x : gc_content_3prime_end(x))
sequence_features.append("ASO_gc_content_3prime_end")

In [167]:
from asodesigner.features.seq_features import homooligo_count
all_data["ASO_homooligo_count"] = all_data["Sequence"].apply(lambda x : homooligo_count(x))
sequence_features.append("ASO_homooligo_count")

Adding  modification features


In [168]:
# compute_mod_fraction
from asodesigner.features.mod_features import compute_mod_fraction

all_data['Modification_fraction'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_fraction(x))
modification_features = ['Modification_fraction']

In [169]:
# compute_mod_type_count
from asodesigner.features.mod_features import compute_mod_type_count

all_data['Modification_type_count'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_type_count(x))
modification_features.append('Modification_type_count')

In [170]:
# compute_mod_5prime_run
from asodesigner.features.mod_features import compute_mod_5prime_run

all_data['Modification_5prime_run'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_5prime_run(x))
modification_features.append('Modification_5prime_run')

In [171]:
# compute_mod_3prime_run
from asodesigner.features.mod_features import compute_mod_3prime_run

all_data['Modification_3prime_run'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_3prime_run(x))
modification_features.append('Modification_3prime_run')

In [172]:
#compute_mod_min_distance_to_5prime
from asodesigner.features.mod_features import compute_mod_min_distance_to_5prime

all_data['Modification_min_distance_to_5prime'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_min_distance_to_5prime(x))
modification_features.append('Modification_min_distance_to_5prime')

In [173]:
#compute_mod_min_distance_to_3prime
from asodesigner.features.mod_features import compute_mod_min_distance_to_3prime

all_data['Modification_min_distance_to_3prime'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_min_distance_to_3prime(x))
modification_features.append('Modification_min_distance_to_3prime')

In [174]:
#compute_mod_pos_std
from asodesigner.features.mod_features import compute_mod_pos_std

all_data['Modification_pos_std'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_pos_std(x))
modification_features.append('Modification_pos_std')

In [175]:
#compute_mod_block_count
from asodesigner.features.mod_features import compute_mod_block_count

all_data['Modification_block_count'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_block_count(x))
modification_features.append('Modification_block_count')

In [176]:
#compute_mod_max_block_length
from asodesigner.features.mod_features import compute_mod_max_block_length

all_data['Modification_max_block_length'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_max_block_length(x))
modification_features.append('Modification_max_block_length')

In [177]:
#compute_mod_char_entropy
from asodesigner.features.mod_features import compute_mod_char_entropy

all_data['Modification_char_entropy'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_char_entropy(x))
modification_features.append('Modification_char_entropy')

In [178]:
#compute_dominant_mod_fraction
from asodesigner.features.mod_features import compute_dominant_mod_fraction

all_data['Modification_dominant_mod_fraction'] = all_data["Chemical_Pattern"].apply(lambda x: compute_dominant_mod_fraction(x))
modification_features.append('Modification_dominant_mod_fraction')

In [179]:
#compute_mod_evenness
from asodesigner.features.mod_features import compute_mod_evenness

all_data['Modification_evenness'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_evenness(x))
modification_features.append('Modification_evenness')

In [180]:
#compute_mod_symmetry_score
from asodesigner.features.mod_features import compute_mod_symmetry_score

all_data['Modification_symmetry_score'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_symmetry_score(x))
modification_features.append('Modification_symmetry_score')

In [181]:
#compute_mod_skew_index
from asodesigner.features.mod_features import compute_mod_skew_index

all_data['Modification_skew_index'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_skew_index(x))
modification_features.append('Modification_skew_index')

In [182]:
#compute_mod_mean_gap
from asodesigner.features.mod_features import compute_mod_mean_gap

all_data['Modification_mean_gap'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_mean_gap(x))
modification_features.append('Modification_mean_gap')

In [183]:
#compute_mod_local_density_max
from asodesigner.features.mod_features import compute_mod_local_density_max

all_data['Modification_local_density_max'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_local_density_max(x))
modification_features.append('Modification_local_density_max')

In [184]:
#compute_mod_in_core
from asodesigner.features.mod_features import compute_mod_in_core

all_data['Modification_in_core'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_in_core(x))
modification_features.append('Modification_in_core')

In [185]:
#compute_mod_longest_repeat_run
from asodesigner.features.mod_features import compute_mod_longest_repeat_run

all_data['Modification_longest_repeat_run'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_longest_repeat_run(x))
modification_features.append('Modification_longest_repeat_run')

In [186]:
#compute_mod_adjacent_pair_count
from asodesigner.features.mod_features import compute_mod_adjacent_pair_count

all_data['Modification_adjacent_pair_count'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_adjacent_pair_count(x))
modification_features.append('Modification_adjacent_pair_count')

In [ ]:
#compute_mod_strong_repeat_group_count
from src.tauso.features.mod_features import compute_mod_strong_repeat_group_count

all_data['Modification_strong_repeat_group_count'] = all_data["Chemical_Pattern"].apply(lambda x: compute_mod_strong_repeat_group_count(x))
modification_features.append('Modification_strong_repeat_group_count')

clean the data

In [188]:
import numpy as np

all_data_no_nan = all_data.dropna(subset=[INHIBITION]).copy()
all_data_no_nan.loc[:, 'log_inhibition'] = -np.log(-all_data_no_nan[INHIBITION] + 100.001)

In [189]:
all_data_no_nan_human = all_data_no_nan[all_data_no_nan[CELL_LINE_ORGANISM] == 'human']
genes = all_data_no_nan[CANONICAL_GENE].copy()
genes_u = list(set(genes))
genes_u.remove('HBV')
genes_u.remove('negative_control')

In [190]:
from asodesigner.read_human_genome import get_locus_to_data_dict
import pickle
from asodesigner.consts import CACHE_DIR


cache_path = CACHE_DIR / 'gene_to_data_simple_cache.pickle'
if not cache_path.exists():
    gene_to_data = get_locus_to_data_dict(include_introns=True, gene_subset=genes_u)
    with open(cache_path, 'wb') as f:
        pickle.dump(gene_to_data, f)
else:
    with open(cache_path, 'rb') as f:
        gene_to_data = pickle.load(f)

In [191]:
# =========================
# External mRNA integration
# =========================
import pandas as pd
from pathlib import Path
import re

def _norm_rna_to_dna(seq: str) -> str:
    """Normalize RNA to DNA alphabet (U->T), uppercase, strip whitespace."""
    return str(seq).upper().replace('U', 'T').replace(' ', '').replace('\t', '').replace('\n', '')

def load_mrna_by_gene_from_files(files: list[str | Path],seq_column: str = "Original Transcript Sequence" ) -> dict[str, str]:
    """
    Load {Gene -> <seq_column>} from a manual list of CSV paths.
    - Expects columns: 'Gene' and <seq_column>
    - Returns DNA alphabet (A/C/G/T) after U->T via _norm_rna_to_dna
    - If multiple rows per gene: keeps the *longest* sequence
    """
    files = [Path(f) for f in files]
    rows = []
    for f in files:
        df = pd.read_csv(f, usecols=['Gene', seq_column])
        df[seq_column] = df[seq_column].map(_norm_rna_to_dna)
        # Keep only clean sequences
        df = df[df[seq_column].str.fullmatch(r'[ACGT]+', na=False)]
        rows.append(df)

    if not rows:
        return {}

    big = pd.concat(rows, ignore_index=True)
    big['len'] = big[seq_column].str.len()
    # Pick longest per gene
    chosen = big.sort_values(['Gene', 'len'], ascending=[True, False]).drop_duplicates('Gene')
    return dict(zip(chosen['Gene'], chosen[seq_column]))

# ---- Choose which mRNA to use for mRNA-based features (tAI/windows on mRNA, etc.) ----
def choose_preferred_mrna(gene_name: str, mrna_built_from_exons: str, gene_to_mrna_real: dict[str,str]) -> str:
    """
    Prefer the real (external) mRNA when available; otherwise fall back to exon-joined.
    Does NOT touch your genome->(mRNA/CDS) mappings or pre-mRNA flanks.
    """
    ext = gene_to_mrna_real.get(gene_name)
    return ext if ext else mrna_built_from_exons


In [192]:
from pathlib import Path
import pandas as pd

DATA_mRNA_PATH = PROJECT_PATH / "scripts" / "data_genertion" / "cell_line_expression"

FILENAMES = [
    "ACH-000232_transcriptome.csv",
    "ACH-000463_transcriptome.csv",
    "ACH-000739_transcriptome.csv",
    "ACH-001086_transcriptome.csv",
    "ACH-001188_transcriptome.csv",
    "ACH-001328_transcriptome.csv",
]

EXTERNAL_MRNA_FILES = [DATA_mRNA_PATH / fn for fn in FILENAMES]
missing = [p.name for p in EXTERNAL_MRNA_FILES if not p.exists()]
assert not missing, f"Missing files in {DATA_mRNA_PATH}: {missing}"

# 1) Load Original
try:
    d_orig = load_mrna_by_gene_from_files(
        [str(p) for p in EXTERNAL_MRNA_FILES],
        seq_column="Original Transcript sequence"
    )
except ValueError:
    # in case the S is capitalized in your headers
    d_orig = load_mrna_by_gene_from_files(
        [str(p) for p in EXTERNAL_MRNA_FILES],
        seq_column="Original Transcript Sequence"
    )

# 2) Load Mutated
try:
    d_mut = load_mrna_by_gene_from_files(
        [str(p) for p in EXTERNAL_MRNA_FILES],
        seq_column="Mutated Transcript sequence"
    )
except ValueError:
    d_mut = load_mrna_by_gene_from_files(
        [str(p) for p in EXTERNAL_MRNA_FILES],
        seq_column="Mutated Transcript Sequence"
    )

# 3) Prefer Mutated when available
gene_to_mrna_real = {**d_orig, **d_mut}

print(f"Loaded {len(gene_to_mrna_real)} real mRNA sequences (Gene -> mRNA).")



Loaded 17546 real mRNA sequences (Gene -> mRNA).


In [193]:
from asodesigner.util import get_antisense
import numpy as np

# Filter relevant genes
all_data_human_gene = all_data_no_nan_human[
    all_data_no_nan_human[CANONICAL_GENE].isin(genes_u)
].copy()

# Column names
SENSE_START       = 'sense_start'          # Genomic index of ASO binding site in pre-mRNA
SENSE_LENGTH      = 'sense_length'         # Length of the ASO (nt)
SENSE_TYPE        = 'sense_type'           # exon / intron
CDS_SEQUENCE      = 'cds_sequence'         # CDS string (joined exons within CDS range)
IN_CODING_REGION  = 'in_coding_region'     # site is within CDS on a real exon

# Flank sizes
FLANK_SIZES_PREMRNA = [20, 30, 40, 50, 60, 70]
FLANK_SIZES_CDS     = [20, 30, 40, 50, 60, 70]

# Initialize columns
all_data_human_gene[SENSE_START]      = 0
all_data_human_gene[SENSE_LENGTH]     = 0
all_data_human_gene[SENSE_TYPE]       = "NA"
all_data_human_gene[CDS_SEQUENCE]     = ""
all_data_human_gene[IN_CODING_REGION] = False

for fs in FLANK_SIZES_PREMRNA:
    all_data_human_gene[f"flank_sequence_{fs}"] = ""
for fs in FLANK_SIZES_CDS:
    all_data_human_gene[f"local_coding_region_around_ASO_{fs}"] = ""

# ---- helpers (local to Part B) ----
def _to_str_seq(x) -> str:
    """
    Coerce sequence-like (list/np.array/Series) or string to a clean uppercase DNA string.
    Converts U->T and strips whitespace. Ensures slicing returns a plain string (avoids pandas iterable assignment).
    """
    if isinstance(x, str):
        s = x
    else:
        try:
            s = ''.join(list(x))
        except Exception:
            s = str(x)
    return s.replace(' ', '').replace('\t', '').replace('\n', '').replace('U', 'T').upper()

def _build_spliced_mrna_from_exons(pre_mrna: str, exon_indices):
    """
    Build exon-joined mRNA by concatenating exon slices out of pre_mrna.
    Keeps your original assumptions: pre_mrna corresponds to genomic strand and
    starts at exon_indices[0][0]; exon intervals are used directly.
    """
    if not exon_indices:
        return ""
    pre_genome_start = exon_indices[0][0]
    parts = []
    for exon_start, exon_end in exon_indices:
        pm_start = exon_start - pre_genome_start
        pm_end   = exon_end   - pre_genome_start
        parts.append(pre_mrna[pm_start:pm_end])
    return "".join(parts)

# Cache CDS per gene
gene_to_cds_info = {}

# ---- main loop ----
for index, row in all_data_human_gene.iterrows():
    gene_name  = row[CANONICAL_GENE]
    locus_info = gene_to_data[gene_name]

    # Keep using your current pre-mRNA for flanks/exon-intron logic (coerced to clean string)
    pre_mrna  = _to_str_seq(locus_info.full_mrna)
    antisense = _to_str_seq(row[SEQUENCE])
    sense     = _to_str_seq(get_antisense(antisense))

    # Locate site on pre-mRNA
    idx = pre_mrna.find(sense)
    all_data_human_gene.at[index, SENSE_START]  = idx
    all_data_human_gene.at[index, SENSE_LENGTH] = len(antisense)

    if idx != -1:
        # Genomic correction (kept as-is)
        genome_corrected_index = idx + locus_info.exon_indices[0][0]

        # exon / intron classification (kept as-is: inclusive check)
        region_type = "intron"
        for exon_start, exon_end in locus_info.exon_indices:
            if exon_start <= genome_corrected_index <= exon_end:
                region_type = "exon"
                break
        all_data_human_gene.at[index, SENSE_TYPE] = region_type

        # pre-mRNA flanks (now using .at and guaranteed string slices)
        for fs in FLANK_SIZES_PREMRNA:
            flank_start = max(0, idx - fs)
            flank_end   = min(len(pre_mrna), idx + len(sense) + fs)
            flank_seq   = pre_mrna[flank_start:flank_end]
            all_data_human_gene.at[index, f"flank_sequence_{fs}"] = flank_seq

        # Build CDS + genome->mRNA map (kept identical to your approach)
        if gene_name not in gene_to_cds_info:
            cds_seq = []  # build as list for speed, join at end
            genome_to_mrna_map = {}
            mrna_idx = 0
            for exon_start, exon_end in locus_info.exon_indices:
                for gpos in range(exon_start, exon_end):
                    if mrna_idx >= len(pre_mrna):
                        break
                    if locus_info.cds_start <= gpos <= locus_info.cds_end:
                        cds_seq.append(pre_mrna[mrna_idx])
                        genome_to_mrna_map[gpos] = len(cds_seq) - 1
                    mrna_idx += 1
            cds_seq = ''.join(cds_seq)
            gene_to_cds_info[gene_name] = (cds_seq, genome_to_mrna_map)
        else:
            cds_seq, genome_to_mrna_map = gene_to_cds_info[gene_name]

        # Save CDS
        all_data_human_gene.at[index, CDS_SEQUENCE] = _to_str_seq(cds_seq)

        #  NEW: prefer real mRNA for mRNA-based features (fallback to exon-joined)
        mrna_built        = _build_spliced_mrna_from_exons(pre_mrna, locus_info.exon_indices)
        mrna_for_features = choose_preferred_mrna(gene_name, mrna_built, gene_to_mrna_real)

        # If within CDS, extract local CDS context (unchanged logic; .at + str)
        if (
            locus_info.cds_start <= genome_corrected_index <= locus_info.cds_end
            and genome_corrected_index in genome_to_mrna_map
        ):
            all_data_human_gene.at[index, IN_CODING_REGION] = True
            cds_idx = genome_to_mrna_map[genome_corrected_index]
            for fs in FLANK_SIZES_CDS:
                start = max(0, cds_idx - fs)
                end   = min(len(cds_seq), cds_idx + len(sense) + fs)
                local_seq = cds_seq[start:end]
                all_data_human_gene.at[index, f"local_coding_region_around_ASO_{fs}"] = _to_str_seq(local_seq)



In [194]:
# Filter genes that were not found in genome
all_data_human_gene_premrna = all_data_human_gene[all_data_human_gene[SENSE_START] != -1]



In [195]:

from sklearn.preprocessing import StandardScaler

# Define flank sizes
CDS_WINDOWS = FLANK_SIZES_CDS

# Loop over each flank window size
for flank in CDS_WINDOWS:
    local_col = f"local_coding_region_around_ASO_{flank}"
    is_local_flag_col = f"region_is_local_{flank}"

    # Create the binary flag: 1 if local exists, 0 otherwise
    all_data_human_gene_premrna[is_local_flag_col] = all_data_human_gene_premrna[local_col].apply(
        lambda x: isinstance(x, str) and x.strip() != ""
    ).astype(int)




/tmp/ipykernel_235140/3972251179.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_human_gene_premrna[is_local_flag_col] = all_data_human_gene_premrna[local_col].apply(
/tmp/ipykernel_235140/3972251179.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_human_gene_premrna[is_local_flag_col] = all_data_human_gene_premrna[local_col].apply(
/tmp/ipykernel_235140/3972251179.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [196]:
from asodesigner.features.seq_features import compute_ENC

# Define flank sizes
CDS_WINDOWS = FLANK_SIZES_CDS

# 1. Local ENC for each flank
for flank in CDS_WINDOWS:
    local_col = f"local_coding_region_around_ASO_{flank}"
    enc_col = f"ENC_score_{flank}_CDS"

    all_data_human_gene_premrna[enc_col] = all_data_human_gene_premrna[local_col].apply(compute_ENC)

# 2. Global ENC for full CDS (calculate once)
all_data_human_gene_premrna["ENC_score_global_CDS"] = all_data_human_gene_premrna["cds_sequence"].apply(compute_ENC)



/tmp/ipykernel_235140/3833547566.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_human_gene_premrna[enc_col] = all_data_human_gene_premrna[local_col].apply(compute_ENC)
/tmp/ipykernel_235140/3833547566.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_human_gene_premrna[enc_col] = all_data_human_gene_premrna[local_col].apply(compute_ENC)
/tmp/ipykernel_235140/3833547566.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [197]:
from asodesigner.features.tai import calc_tAI, tai_weights

# Load tAI weights for human
weights_tai = tai_weights("hm")

# 1. Local tAI for each flank
for flank in CDS_WINDOWS:
    local_col = f"local_coding_region_around_ASO_{flank}"
    tai_col = f"tAI_score_{flank}_CDS"

    def safe_calc_tAI(seq):
        if not isinstance(seq, str) or len(seq.strip()) == 0:
            return np.nan
        return calc_tAI(seq, weights_tai)

    all_data_human_gene_premrna[tai_col] = all_data_human_gene_premrna[local_col].apply(safe_calc_tAI)

# 2. Global tAI for full CDS
def safe_calc_tAI_global(seq):
    if not isinstance(seq, str) or len(seq.strip()) == 0:
        return np.nan
    return calc_tAI(seq, weights_tai)

all_data_human_gene_premrna["tAI_score_global_CDS"] = all_data_human_gene_premrna["cds_sequence"].apply(safe_calc_tAI_global)



/tmp/ipykernel_235140/2329384670.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_human_gene_premrna[tai_col] = all_data_human_gene_premrna[local_col].apply(safe_calc_tAI)
/tmp/ipykernel_235140/2329384670.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_human_gene_premrna[tai_col] = all_data_human_gene_premrna[local_col].apply(safe_calc_tAI)
/tmp/ipykernel_235140/2329384670.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [198]:

from sklearn.preprocessing import StandardScaler


feature_func_map = {
    "gc_content": gc_fraction,
    "purine_content": purine_content,
    "sequence_entropy": seq_entropy,
    "homooligo_count": homooligo_count,
    "entropy": seq_entropy,
    'ggg_counts': count_g_runs,
    "toxic_motif_count": toxic_motif_count,
    "tandem_repeats_score": tandem_repeats_score,
    "dispersed_repeats_score": dispersed_repeats_score,
    "flexible_dinucleotide_fraction": flexible_dinucleotide_fraction,
    "gc_skew": gc_skew,
    "gc_skew_ends": gc_skew_ends,
    "at_skew": at_skew,
    "stop_codon_count": stop_codon_count,
    "nucleotide_diversity": nucleotide_diversity,
    "cg_dinucleotide_fraction": cg_dinucleotide_fraction,
    "poly_pyrimidine_stretch": poly_pyrimidine_stretch,
    "dinucleotide_entropy": dinucleotide_entropy,
    "gc_block_length": gc_block_length,
    "at_rich_region_score": at_rich_region_score,
    "gc_content_3prime_end": gc_content_3prime_end

}

pre_mRNA_features = []

for flank in FLANK_SIZES_PREMRNA:
    sequence_column = f"flank_sequence_{flank}"

    for feature_name, func in feature_func_map.items():
        output_column = f"premRNA_{feature_name}_{flank}"
        all_data_human_gene_premrna[output_column] = all_data_human_gene_premrna[sequence_column].apply(func)
        pre_mRNA_features.append(output_column)

for flank in FLANK_SIZES_PREMRNA:
    sequence_column = f"flank_sequence_{flank}"
    pal_4_column = f"premRNA_4 palindromic_{flank}"
    pal_6_column = f"premRNA_6 palindromic_{flank}"
    all_data_human_gene_premrna[pal_4_column] =all_data_human_gene_premrna[sequence_column].apply(lambda x: palindromic_fraction(x, 4))
    all_data_human_gene_premrna[pal_6_column] =all_data_human_gene_premrna[sequence_column].apply(lambda x: palindromic_fraction(x, 6))
    pre_mRNA_features.append(pal_4_column)
    pre_mRNA_features.append(pal_6_column)


/tmp/ipykernel_235140/1054432525.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_human_gene_premrna[output_column] = all_data_human_gene_premrna[sequence_column].apply(func)
/tmp/ipykernel_235140/1054432525.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_human_gene_premrna[output_column] = all_data_human_gene_premrna[sequence_column].apply(func)
/tmp/ipykernel_235140/1054432525.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [199]:
"""""
# 6. Normalize each feature using Z-score, separately for local vs global regions
from sklearn.preprocessing import StandardScaler

# Flank sizes (CDS-based regions)
CDS_WINDOWS = FLANK_SIZES_CDS

# Base feature names to normalize
FEATURE_BASE_NAMES = ["ENC_score", "tAI_score", "chimera_score_CDS"]

for flank in CDS_WINDOWS:
    local_mask = all_data_human_gene_premrna[f"region_is_local_{flank}"] == 1
    global_mask = ~local_mask

    for base_feature in FEATURE_BASE_NAMES:
        feature_col = f"{base_feature}_{flank}"

        # Normalize local values (region was local CDS context)
        scaler_local = StandardScaler()
        all_data_human_gene_premrna.loc[local_mask, feature_col] = scaler_local.fit_transform(
            all_data_human_gene_premrna.loc[local_mask, [feature_col]]
        )

        # Normalize global values (region was full CDS fallback)
        scaler_global = StandardScaler()
        all_data_human_gene_premrna.loc[global_mask, feature_col] = scaler_global.fit_transform(
            all_data_human_gene_premrna.loc[global_mask, [feature_col]]
        )


"""

'""\n# 6. Normalize each feature using Z-score, separately for local vs global regions\nfrom sklearn.preprocessing import StandardScaler\n\n# Flank sizes (CDS-based regions)\nCDS_WINDOWS = FLANK_SIZES_CDS\n\n# Base feature names to normalize\nFEATURE_BASE_NAMES = ["ENC_score", "tAI_score", "chimera_score_CDS"]\n\nfor flank in CDS_WINDOWS:\n    local_mask = all_data_human_gene_premrna[f"region_is_local_{flank}"] == 1\n    global_mask = ~local_mask\n\n    for base_feature in FEATURE_BASE_NAMES:\n        feature_col = f"{base_feature}_{flank}"\n\n        # Normalize local values (region was local CDS context)\n        scaler_local = StandardScaler()\n        all_data_human_gene_premrna.loc[local_mask, feature_col] = scaler_local.fit_transform(\n            all_data_human_gene_premrna.loc[local_mask, [feature_col]]\n        )\n\n        # Normalize global values (region was full CDS fallback)\n        scaler_global = StandardScaler()\n        all_data_human_gene_premrna.loc[global_mask, fe

In [200]:
all_data_human_gene_premrna.head(
    
)

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,premRNA_4 palindromic_30,premRNA_6 palindromic_30,premRNA_4 palindromic_40,premRNA_6 palindromic_40,premRNA_4 palindromic_50,premRNA_6 palindromic_50,premRNA_4 palindromic_60,premRNA_6 palindromic_60,premRNA_4 palindromic_70,premRNA_6 palindromic_70
0,0,540733,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCTAAAACAAATGCTA,...,0.078947,0.013158,0.093750,0.010417,0.094828,0.017241,0.080882,0.022059,0.089744,0.025641
1,1,540747,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,TATAATGGTGAATATC,...,0.092105,0.013158,0.083333,0.010417,0.086207,0.017241,0.073529,0.014706,0.076923,0.012821
2,2,540806,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCATGAAGATTTCTGG,...,0.078947,0.026316,0.062500,0.020833,0.060345,0.017241,0.058824,0.014706,0.057692,0.019231
3,3,651479,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GGTGAATATCTTCAAA,...,0.078947,0.013158,0.072917,0.010417,0.068966,0.008621,0.088235,0.014706,0.076923,0.012821
4,4,651490,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,CACTTGTACTAGTATG,...,0.105263,0.039474,0.093750,0.041667,0.077586,0.034483,0.073529,0.029412,0.064103,0.025641


In [201]:
all_data_human_gene_premrna.columns

Index(['index', 'ISIS', 'Target_gene', 'Cell_line', 'Density(cells/well)',
       'Transfection', 'ASO_volume(nM)', 'Treatment_Period(hours)',
       'Primer_probe_set', 'Sequence',
       ...
       'premRNA_4 palindromic_30', 'premRNA_6 palindromic_30',
       'premRNA_4 palindromic_40', 'premRNA_6 palindromic_40',
       'premRNA_4 palindromic_50', 'premRNA_6 palindromic_50',
       'premRNA_4 palindromic_60', 'premRNA_6 palindromic_60',
       'premRNA_4 palindromic_70', 'premRNA_6 palindromic_70'],
      dtype='object', length=249)

In [202]:
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection  import train_test_split, cross_val_score
from sklearn.metrics import make_scorer
import pandas as pd
from scripts.data_genertion.consts import *

In [203]:
tAI_list = [f'tAI_score_{flank}_CDS' for flank in FLANK_SIZES_CDS] + ['tAI_score_global_CDS']
ENC_list = [f'ENC_score_{flank}_CDS' for flank in FLANK_SIZES_CDS] + ['ENC_score_global_CDS']
region_is_local_list = [f"region_is_local_{flank}" for flank in FLANK_SIZES_CDS]
 

In [204]:
all_data_human_gene_premrna_no_nan = all_data_human_gene_premrna.copy()
filtered = all_data_human_gene_premrna_no_nan
filtered = filtered[filtered['Cell line organism'] == 'human']
filtered = filtered[filtered['Cell line organism'] == 'human']
print(len(filtered))
filtered = filtered.dropna(subset=[INHIBITION]).copy()
filtered = filtered.dropna(subset=[DENSITY]).copy()

ONE_HOT_FEATURE_NAMES = [CELL_LINE, TRANSFECTION, MODIFICATION, SENSE_TYPE]
# append more one-hot features 'first_nucleotide', 'second_nucleotide'

for one_hot_feature in ONE_HOT_FEATURE_NAMES:
    filtered = pd.get_dummies(filtered, columns=[one_hot_feature]).copy()

filtered.loc[:, 'log_volume'] = np.log(filtered[VOLUME])
filtered.loc[:, 'log_density'] = np.log(filtered[DENSITY])

# more feature generation
feature_base = [SENSE_START, SENSE_LENGTH]
experiment_features = [VOLUME, 'log_volume', 'log_density', TREATMENT_PERIOD]

one_hot_encoded_features = []
for one_hot_feature in ONE_HOT_FEATURE_NAMES:
    one_hot_encoded_features.append([feature for feature in filtered.columns if one_hot_feature in feature and one_hot_feature != feature])

# Flatten the list of list to a single big list
flat_one_hot_encoded_features = [feature for sublist in one_hot_encoded_features for feature in sublist]

features = feature_base + experiment_features + flat_one_hot_encoded_features + sequence_features + modification_features +pre_mRNA_features +tAI_list + ENC_list + region_is_local_list 

29987


/home/nivd1/miniconda3/envs/aso_design/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [205]:
import re
import numpy as np
#This code identifies all local CDS window columns, replaces empty strings with NaN, and then creates a new flag column aso_in_CDS.
#The flag is True if an ASO overlaps at least one CDS window, and False otherwise.

def list_local_cols(df):
    pat = re.compile(r"^local_coding_region_around_ASO_")
    return [c for c in df.columns if pat.match(c)]

LOCAL_COLS = list_local_cols(filtered)
filtered[LOCAL_COLS] = filtered[LOCAL_COLS].replace("", np.nan)

filtered["aso_in_CDS"] = filtered[LOCAL_COLS].notna().any(axis=1)

Remove the sequence columns

In [206]:
remove_flank_seq_list = [col for col in filtered.columns if col.startswith("flank_sequence_")]
remove_local_coding_region_list = [col for col in filtered.columns if col.startswith("local_coding_region_around_")]
remove_feature_list = [col for col in filtered.columns if col.startswith("region_for_features_")]
drop_list = remove_flank_seq_list + remove_local_coding_region_list + remove_feature_list
new_filtered = filtered.drop(columns=drop_list).copy()

 Save each feature as CSV in the features directory

In [ ]:
""""
from scripts.features.feature_extraction import save_feature
feature_list =sequence_features + modification_features +pre_mRNA_features +tAI_list + ENC_list  + ["aso_in_CDS"]
for feat in feature_list:
    save_feature(new_filtered, feat)
"""


 Save DataFrame as CSV in the current working directory

In [208]:

#new_filtered.to_csv("features_output_final_update_CUB_final2.csv", index=False)